In [2]:
import os
import sys
from pathlib import Path

# Set project root
PROJECT_ROOT = Path.cwd()
os.chdir(PROJECT_ROOT)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
os.chdir(r"D:\Visual Studio practice\aviation-chatbot")
print(f"✅ Project root: {PROJECT_ROOT}")

✅ Project root: d:\Visual Studio practice\aviation-chatbot\notebooks


In [3]:
import psycopg2
import google.generativeai as genai
from sentence_transformers import SentenceTransformer

from src.config import (
    DB_HOST, DB_PORT, DB_NAME, DB_USER, DB_PASSWORD,
    EMBEDDING_MODEL_NAME,
    GEMINI_API_KEY, GEMINI_MODEL,
    TOP_K_RETRIEVAL,
    get_rag_prompt
)

print("✅ All libraries imported successfully")
print(f"\n🤖 Embedding Model: {EMBEDDING_MODEL_NAME}")
print(f"🌟 Gemini Model: {GEMINI_MODEL}")
print(f"📊 Retrieval K: {TOP_K_RETRIEVAL}")

d:\Visual Studio practice\aviation-chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\kumme\AppData\Local\Temp\ipykernel_29736\26551832.py:2: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


D:\Visual Studio practice\aviation-chatbot
✅ All libraries imported successfully

🤖 Embedding Model: all-MiniLM-L6-v2
🌟 Gemini Model: gemini-2.5-flash
📊 Retrieval K: 5


In [4]:
# Load embedding model for query encoding
print("🔄 Loading embedding model...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)
print("✅ Embedding model loaded\n")

# Configure Gemini API
print("🔄 Configuring Gemini API...")
genai.configure(api_key=GEMINI_API_KEY)
print("✅ Gemini API configured\n")

# Use the latest available model
MODEL_NAME = "gemini-2.5-flash"  # Fast and efficient!

print(f"🔄 Initializing model: {MODEL_NAME}...")
gemini_model = genai.GenerativeModel(MODEL_NAME)
print("✅ Model initialized\n")

# Test Gemini connection
print("🧪 Testing Gemini connection...")
try:
    test_response = gemini_model.generate_content("Hello! Just testing the connection. Reply with 'OK' if you receive this.")
    print("✅ Gemini connection successful!")
    print(f"   Model: {MODEL_NAME}")
    print(f"   Response: {test_response.text[:100]}...")
except Exception as e:
    print(f"❌ Gemini connection failed: {e}")

🔄 Loading embedding model...


d:\Visual Studio practice\aviation-chatbot\.venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Embedding model loaded

🔄 Configuring Gemini API...
✅ Gemini API configured

🔄 Initializing model: gemini-2.5-flash...
✅ Model initialized

🧪 Testing Gemini connection...
✅ Gemini connection successful!
   Model: gemini-2.5-flash
   Response: OK...


In [7]:
def retrieve_chunks(query, top_k=TOP_K_RETRIEVAL):
    """
    Retrieve most relevant chunks from PostgreSQL using vector similarity
    
    Args:
        query: User's question
        top_k: Number of chunks to retrieve
    
    Returns:
        List of tuples: (content, document_name, page_number, similarity_score)
    """
    # Generate query embedding
    query_embedding = embedding_model.encode([query])[0]
    
    # Connect to database
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = conn.cursor()
    
    # Perform similarity search using cosine distance
    cursor.execute("""
        SELECT 
            content,
            document_name,
            page_number,
            1 - (embedding <=> %s::vector) as similarity
        FROM knowledge_chunks
        ORDER BY embedding <=> %s::vector
        LIMIT %s;
    """, (query_embedding.tolist(), query_embedding.tolist(), top_k))
    
    results = cursor.fetchall()
    
    cursor.close()
    conn.close()
    
    return results

# Test retrieval
print("🔍 TESTING RETRIEVAL FUNCTION")
print("=" * 80)

test_query = "What is SCADA?"
print(f"\nQuery: '{test_query}'\n")

results = retrieve_chunks(test_query, top_k=3)

print(f"✅ Retrieved {len(results)} chunks:\n")
for i, (content, doc_name, page_num, similarity) in enumerate(results, 1):
    print(f"{i}. [{doc_name} | Page {page_num}] (Similarity: {similarity:.4f})")
    print(f"   {content[:200]}...\n")

print("=" * 80)

🔍 TESTING RETRIEVAL FUNCTION

Query: 'What is SCADA?'

✅ Retrieved 3 chunks:

1. [scada_manual.pdf | Page 8] (Similarity: 0.7020)
   Preface to the Third Edition
When the first edition of this book was written, certain trends in SCADA
were already apparent, and I made attempts to identify them in Unit 14,
"What's Next?". Generally,...

2. [scada_manual.pdf | Page 13] (Similarity: 0.6681)
   electronics-based technologies, SCAD A is a virtual cornucopia of these
terms and abbreviations. Finally, the solutions to the exercises found at the
end of each unit are given in Appendix C.
1-6. Cou...

3. [scada_manual.pdf | Page 145] (Similarity: 0.6465)
   respectively. Very often people confuse the terms in the mistaken belief
that because SCAD A is associated with so much expensive computer
hardware it must be automatic. In fact, most early SCADA syst...



In [8]:
def generate_answer(query, retrieved_chunks):
    """
    Generate answer using Gemini LLM with retrieved context
    
    Args:
        query: User's question
        retrieved_chunks: List of (content, doc_name, page_num, similarity)
    
    Returns:
        Generated answer string
    """
    # Format context from retrieved chunks
    context_parts = []
    for content, doc_name, page_num, similarity in retrieved_chunks:
        context_parts.append(
            f"[Source: {doc_name}, Page {page_num}, Relevance: {similarity:.2f}]\n{content}"
        )
    
    context = "\n\n---\n\n".join(context_parts)
    
    # Build prompt using config template
    prompt = get_rag_prompt(query, context)
    
    # Generate answer with Gemini
    try:
        response = gemini_model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"❌ Error generating answer: {e}"

# Test answer generation
print("🤖 TESTING ANSWER GENERATION")
print("=" * 80)

test_query = "What is SCADA?"
print(f"\n❓ Question: {test_query}\n")

print("🔄 Retrieving context...")
retrieved = retrieve_chunks(test_query, top_k=5)
print(f"✅ Retrieved {len(retrieved)} chunks\n")

print("🔄 Generating answer with Gemini...\n")
answer = generate_answer(test_query, retrieved)

print("💡 ANSWER:")
print("-" * 80)
print(answer)
print("-" * 80)

print("\n✅ Answer generation successful!")

🤖 TESTING ANSWER GENERATION

❓ Question: What is SCADA?

🔄 Retrieving context...
✅ Retrieved 5 chunks

🔄 Generating answer with Gemini...

💡 ANSWER:
--------------------------------------------------------------------------------
Based on the provided context, SCADA can be described as follows:

SCADA refers to an electronics-based technology (Source: scada_manual.pdf, Page 13) associated with significant computer hardware (Source: scada_manual.pdf, Page 145). Its primary function is as a very efficient remote control and remote data-gathering system (Source: scada_manual.pdf, Page 145), used to control and monitor processes (Source: scada_manual.pdf, Page 47).

It is important to note that while SCADA is often associated with automation, most early SCADA systems and a large minority of present ones are not automatic, frequently requiring intervention (Source: scada_manual.pdf, Page 145). SCADA projects are complex, involving a large number of elements, and require substantial engineer

In [9]:
def aviation_chatbot(query, top_k=5, show_sources=True, show_context=False):
    """
    Complete RAG pipeline: Retrieve → Generate Answer
    
    Args:
        query: User's question
        top_k: Number of chunks to retrieve
        show_sources: Whether to display source information
        show_context: Whether to display retrieved context
    
    Returns:
        Generated answer
    """
    print("=" * 80)
    print(f"🛫 AVIATION CHATBOT")
    print("=" * 80)
    print(f"\n❓ Your Question:\n   {query}\n")
    
    # Step 1: Retrieve relevant chunks
    print(f"🔍 Retrieving top {top_k} relevant chunks...")
    retrieved_chunks = retrieve_chunks(query, top_k=top_k)
    print(f"✅ Retrieved {len(retrieved_chunks)} chunks\n")
    
    # Show sources if requested
    if show_sources:
        print("📚 Sources:")
        for i, (content, doc_name, page_num, similarity) in enumerate(retrieved_chunks, 1):
            print(f"   {i}. {doc_name} (Page {page_num}) - Relevance: {similarity:.3f}")
        print()
    
    # Show context if requested
    if show_context:
        print("📄 Retrieved Context:")
        print("-" * 80)
        for i, (content, doc_name, page_num, similarity) in enumerate(retrieved_chunks, 1):
            print(f"\n[Chunk {i}] {doc_name}, Page {page_num}")
            print(content[:300] + "...\n")
        print("-" * 80)
        print()
    
    # Step 2: Generate answer
    print("🤖 Generating answer with Gemini...\n")
    answer = generate_answer(query, retrieved_chunks)
    
    print("💡 ANSWER:")
    print("-" * 80)
    print(answer)
    print("-" * 80)
    print("\n✅ Query completed!\n")
    
    return answer

# Test the complete pipeline
print("🧪 TESTING COMPLETE RAG PIPELINE\n")

answer = aviation_chatbot(
    "What is SCADA and how does it work?", 
    top_k=5, 
    show_sources=True,
    show_context=False
)

🧪 TESTING COMPLETE RAG PIPELINE

🛫 AVIATION CHATBOT

❓ Your Question:
   What is SCADA and how does it work?

🔍 Retrieving top 5 relevant chunks...
✅ Retrieved 5 chunks

📚 Sources:
   1. scada_manual.pdf (Page 8) - Relevance: 0.683
   2. scada_manual.pdf (Page 13) - Relevance: 0.654
   3. scada_manual.pdf (Page 145) - Relevance: 0.644
   4. scada_manual.pdf (Page 47) - Relevance: 0.600
   5. scada_manual.pdf (Page 197) - Relevance: 0.588

🤖 Generating answer with Gemini...

💡 ANSWER:
--------------------------------------------------------------------------------
Based on the provided context:

SCADA (Supervisory Control and Data Acquisition) systems are associated with significant computer hardware and are primarily designed as very efficient remote control and remote data-gathering systems. While often confused with fully automatic systems, many early SCADA systems and a large minority of present ones are not automatic, often requiring human intervention (Source: scada_manual.pdf, Pa

In [10]:
import time

# Test various aviation-related questions
test_queries = [
    "What are the responsibilities of airport apron control?",
    "How does SCADA help in airport operations?",
    "What are the main components of airport infrastructure?",
]

print("🧪 TESTING MULTIPLE QUERIES")
print("=" * 80)
print(f"\nRunning {len(test_queries)} test queries...\n")

for i, query in enumerate(test_queries, 1):
    print(f"\n\n{'='*80}")
    print(f"TEST {i}/{len(test_queries)}")
    print(f"{'='*80}\n")
    
    try:
        answer = aviation_chatbot(query, top_k=3, show_sources=True, show_context=False)
        
        # Add delay to respect API rate limits (Gemini free tier: 15 requests/min)
        if i < len(test_queries):
            print(f"\n⏳ Waiting 5 seconds before next query...")
            time.sleep(5)
    except Exception as e:
        print(f"❌ Error on query {i}: {e}")
        continue

print("\n\n" + "="*80)
print("✅ All tests completed!")
print("="*80)

🧪 TESTING MULTIPLE QUERIES

Running 3 test queries...



TEST 1/3

🛫 AVIATION CHATBOT

❓ Your Question:
   What are the responsibilities of airport apron control?

🔍 Retrieving top 3 relevant chunks...
✅ Retrieved 3 chunks

📚 Sources:
   1. airport_operations.pdf (Page 244) - Relevance: 0.610
   2. airport_operations.pdf (Page 154) - Relevance: 0.598
   3. airport_operations.pdf (Page 463) - Relevance: 0.570

🤖 Generating answer with Gemini...

💡 ANSWER:
--------------------------------------------------------------------------------
Based on the provided context, the specific responsibilities of "airport apron control" are not explicitly defined.

However, the context does outline several safety measures that are implemented on the apron:
*   Provision of suitable firefighting equipment (Source: airport_operations.pdf, Page 154)
*   Provision of other necessary protective equipment (Source: airport_operations.pdf, Page 154)
*   Provision of security personnel where required (Source: a

In [11]:
def ask_question(query, top_k=5):
    """
    Simple wrapper for single question queries
    
    Args:
        query: User's question
        top_k: Number of chunks to retrieve
    
    Returns:
        Generated answer
    """
    return aviation_chatbot(query, top_k=top_k, show_sources=True, show_context=False)

# Try asking different questions
print("💬 INTERACTIVE TESTING\n")
print("You can now ask questions using: ask_question('your question here')\n")
print("Example queries to try:")
print("  - ask_question('What are runway safety procedures?')")
print("  - ask_question('Explain airport terminal design')")
print("  - ask_question('What is the role of air traffic control?')")
print("\n" + "="*80)

# Uncomment below to test
# ask_question("What are runway safety procedures?")

💬 INTERACTIVE TESTING

You can now ask questions using: ask_question('your question here')

Example queries to try:
  - ask_question('What are runway safety procedures?')
  - ask_question('Explain airport terminal design')
  - ask_question('What is the role of air traffic control?')



In [12]:
def evaluate_retrieval_quality(query, expected_docs=None, top_k=10):
    """
    Evaluate retrieval quality for a given query
    
    Args:
        query: Test query
        expected_docs: List of expected document names (optional)
        top_k: Number of chunks to retrieve
    """
    print(f"\n📊 RETRIEVAL QUALITY ANALYSIS")
    print("=" * 80)
    print(f"Query: {query}\n")
    
    # Retrieve chunks
    chunks = retrieve_chunks(query, top_k=top_k)
    
    # Analyze results
    doc_distribution = {}
    similarities = []
    page_distribution = {}
    
    for content, doc_name, page_num, similarity in chunks:
        similarities.append(similarity)
        
        # Document distribution
        if doc_name not in doc_distribution:
            doc_distribution[doc_name] = []
        doc_distribution[doc_name].append((page_num, similarity))
        
        # Page distribution
        page_key = f"{doc_name}:p{page_num}"
        if page_key not in page_distribution:
            page_distribution[page_key] = 0
        page_distribution[page_key] += 1
    
    # Display metrics
    print(f"📈 Similarity Score Statistics:")
    print(f"   Maximum:  {max(similarities):.4f}")
    print(f"   Minimum:  {min(similarities):.4f}")
    print(f"   Average:  {sum(similarities)/len(similarities):.4f}")
    print(f"   Range:    {max(similarities) - min(similarities):.4f}\n")
    
    print(f"📚 Document Distribution:")
    for doc_name, pages in doc_distribution.items():
        print(f"   📄 {doc_name}: {len(pages)} chunks")
        avg_score = sum(s for _, s in pages) / len(pages)
        print(f"      Average relevance: {avg_score:.4f}")
        unique_pages = sorted(set(p for p, _ in pages))
        print(f"      Pages covered: {unique_pages[:5]}" + 
              (f" ... (+{len(unique_pages)-5} more)" if len(unique_pages) > 5 else ""))
        print()
    
    # Check if results match expected documents
    if expected_docs:
        retrieved_docs = set(doc_distribution.keys())
        expected_set = set(expected_docs)
        
        if expected_set.issubset(retrieved_docs):
            print(f"✅ All expected documents found in results")
        else:
            missing = expected_set - retrieved_docs
            print(f"⚠️ Missing expected documents: {missing}")
    
    print("=" * 80)

# Test retrieval quality for different queries
print("🧪 TESTING RETRIEVAL QUALITY\n")

evaluate_retrieval_quality("What is SCADA?", expected_docs=["scada_manual.pdf"])
print("\n")
evaluate_retrieval_quality("Airport runway design standards", expected_docs=["airport_operations.pdf"])

🧪 TESTING RETRIEVAL QUALITY


📊 RETRIEVAL QUALITY ANALYSIS
Query: What is SCADA?

📈 Similarity Score Statistics:
   Maximum:  0.7020
   Minimum:  0.5468
   Average:  0.6064
   Range:    0.1553

📚 Document Distribution:
   📄 scada_manual.pdf: 10 chunks
      Average relevance: 0.6064
      Pages covered: [8, 13, 22, 24, 28] ... (+5 more)

✅ All expected documents found in results



📊 RETRIEVAL QUALITY ANALYSIS
Query: Airport runway design standards

📈 Similarity Score Statistics:
   Maximum:  0.6052
   Minimum:  0.5018
   Average:  0.5381
   Range:    0.1034

📚 Document Distribution:
   📄 airport_operations.pdf: 10 chunks
      Average relevance: 0.5381
      Pages covered: [109, 144, 145, 242, 317] ... (+2 more)

✅ All expected documents found in results


In [14]:
import json
from datetime import datetime
from pathlib import Path

def save_query_log(query, answer, sources, retrieval_time=None, generation_time=None):
    """
    Save query history to JSON file for analysis
    
    Args:
        query: User's question
        answer: Generated answer
        sources: Retrieved source chunks
        retrieval_time: Time taken for retrieval (optional)
        generation_time: Time taken for generation (optional)
    """
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "query": query,
        "answer": answer,
        "num_sources": len(sources),
        "sources": [
            {
                "document": doc_name,
                "page": page_num,
                "similarity": float(similarity),
                "content_preview": content[:100]
            }
            for content, doc_name, page_num, similarity in sources
        ],
        "performance": {
            "retrieval_time": retrieval_time,
            "generation_time": generation_time
        }
    }
    
    # Ensure data directory exists
    data_dir = PROJECT_ROOT / "data"
    data_dir.mkdir(parents=True, exist_ok=True)
    
    log_file = data_dir / "query_history.json"
    
    # Load existing logs
    if log_file.exists():
        with open(log_file, "r", encoding="utf-8") as f:
            try:
                logs = json.load(f)
            except:
                logs = []
    else:
        logs = []
    
    # Append new log
    logs.append(log_entry)
    
    # Save (keep last 100 queries)
    with open(log_file, "w", encoding="utf-8") as f:
        json.dump(logs[-100:], f, indent=2, ensure_ascii=False)
    
    print(f"✅ Query logged to {log_file}")
    print(f"   Total queries in history: {len(logs)}")

# Example: Log a query with timing
print("📝 TESTING QUERY LOGGING\n")

query = "What are airport safety regulations?"

# Time the retrieval
import time
start_retrieval = time.time()
retrieved = retrieve_chunks(query, top_k=5)
retrieval_time = time.time() - start_retrieval

# Time the generation
start_generation = time.time()
answer = generate_answer(query, retrieved)
generation_time = time.time() - start_generation

# Log the query
save_query_log(query, answer, retrieved, retrieval_time, generation_time)

print(f"\n⏱️ Performance:")
print(f"   Retrieval: {retrieval_time:.3f}s")
print(f"   Generation: {generation_time:.3f}s")
print(f"   Total: {retrieval_time + generation_time:.3f}s")

📝 TESTING QUERY LOGGING

✅ Query logged to d:\Visual Studio practice\aviation-chatbot\notebooks\data\query_history.json
   Total queries in history: 1

⏱️ Performance:
   Retrieval: 0.046s
   Generation: 4.969s
   Total: 5.015s


In [15]:
ask_question("What are the safety procedures for runway operations?")

🛫 AVIATION CHATBOT

❓ Your Question:
   What are the safety procedures for runway operations?

🔍 Retrieving top 5 relevant chunks...
✅ Retrieved 5 chunks

📚 Sources:
   1. airport_operations.pdf (Page 449) - Relevance: 0.661
   2. airport_operations.pdf (Page 500) - Relevance: 0.596
   3. airport_operations.pdf (Page 517) - Relevance: 0.542
   4. airport_operations.pdf (Page 516) - Relevance: 0.540
   5. airport_operations.pdf (Page 433) - Relevance: 0.528

🤖 Generating answer with Gemini...

💡 ANSWER:
--------------------------------------------------------------------------------
Based on the provided context, the safety procedures for runway operations include the following:

1.  **Risk Management for Runway Incursions:** To control risks leading to runway incursions, a detailed work plan is required. This plan should encompass:
    *   Detailed movement routes
    *   Communication protocols
    *   Evacuation procedures
    *   Scheduled briefings
    *   Inspections
    *   Turno

"Based on the provided context, the safety procedures for runway operations include the following:\n\n1.  **Risk Management for Runway Incursions:** To control risks leading to runway incursions, a detailed work plan is required. This plan should encompass:\n    *   Detailed movement routes\n    *   Communication protocols\n    *   Evacuation procedures\n    *   Scheduled briefings\n    *   Inspections\n    *   Turnover procedures\n    *   Specific control measures\n    (Source: airport_operations.pdf, Page 449)\n\n2.  **Operational Procedures:** These procedures are critical for ground movement control and include:\n    *   Rules of access on movement areas.\n    *   Right-of-way guidelines.\n    *   Vehicle obstruction marking.\n    *   A defined structure of control and duties for personnel.\n    *   Routine inspection procedures for movement areas, specifically including runways.\n    (Source: airport_operations.pdf, Page 500)\n\n3.  **Maintenance and Inspection of Paved and Safety

In [16]:
evaluate_retrieval_quality("airport security protocols")


📊 RETRIEVAL QUALITY ANALYSIS
Query: airport security protocols

📈 Similarity Score Statistics:
   Maximum:  0.7088
   Minimum:  0.4873
   Average:  0.5516
   Range:    0.2215

📚 Document Distribution:
   📄 airport_operations.pdf: 10 chunks
      Average relevance: 0.5516
      Pages covered: [9, 119, 241, 242, 243] ... (+5 more)



In [21]:
import json

# Open with UTF-8 encoding
with open("data/query_history.json", "r", encoding="utf-8") as f:
    history = json.load(f)
    print(f"✅ Total queries: {len(history)}")
    
    # Show summary of last 3 queries
    print(f"\n📊 Recent Queries:\n")
    for i, entry in enumerate(history[-3:], 1):
        print(f"{i}. Query: {entry['query']}")
        print(f"   Timestamp: {entry['timestamp']}")
        print(f"   Sources used: {entry['num_sources']}")
        if 'performance' in entry and entry['performance']['retrieval_time']:
            print(f"   Retrieval time: {entry['performance']['retrieval_time']:.3f}s")
            print(f"   Generation time: {entry['performance']['generation_time']:.3f}s")
        print()

✅ Total queries: 1

📊 Recent Queries:

1. Query: What are airport safety regulations?
   Timestamp: 2026-01-21T14:43:17.589996
   Sources used: 5
   Retrieval time: 0.084s
   Generation time: 5.925s



In [22]:
from datetime import datetime

def view_query_history(limit=10):
    """View formatted query history"""
    
    with open("data/query_history.json", "r", encoding="utf-8") as f:
        history = json.load(f)
    
    print("=" * 80)
    print(f"📜 QUERY HISTORY ({len(history)} total queries)")
    print("=" * 80)
    
    # Show last N queries
    recent = history[-limit:]
    
    for i, entry in enumerate(reversed(recent), 1):
        print(f"\n{i}. 📅 {entry['timestamp']}")
        print(f"   ❓ Query: {entry['query']}")
        print(f"   📚 Sources: {entry['num_sources']} chunks from:")
        
        # Show unique documents used
        docs = set(src['document'] for src in entry['sources'])
        for doc in docs:
            print(f"      • {doc}")
        
        # Show performance if available
        if entry.get('performance', {}).get('retrieval_time'):
            perf = entry['performance']
            total = perf['retrieval_time'] + perf['generation_time']
            print(f"   ⏱️  Performance: {total:.2f}s (retrieval: {perf['retrieval_time']:.2f}s, generation: {perf['generation_time']:.2f}s)")
        
        # Show answer preview
        answer_preview = entry['answer'][:150] + "..." if len(entry['answer']) > 150 else entry['answer']
        print(f"   💡 Answer: {answer_preview}")
        print("-" * 80)

# View last 10 queries
view_query_history(limit=10)

📜 QUERY HISTORY (1 total queries)

1. 📅 2026-01-21T14:43:17.589996
   ❓ Query: What are airport safety regulations?
   📚 Sources: 5 chunks from:
      • airport_operations.pdf
   ⏱️  Performance: 6.01s (retrieval: 0.08s, generation: 5.92s)
   💡 Answer: Airport safety regulations are established at both international and national levels to ensure the safe conduct of aviation.

Globally, the foundation...
--------------------------------------------------------------------------------
